In [1]:
from pystac import Item
from odc.stac import load
import odc.geo.xr  # noqa: F401

In [3]:
tiles = [
    (118, -12),
    (119, -12),
    (118, -13),
    (119, -13),
]

items = []

for tile in tiles:
    url = f"https://data.ldn.auspatious.com/test/geo_ls_lp/0_0_1/2023/{tile[0]}/{tile[1]}/geo-ls-lp_2023_{tile[0]}_{tile[1]}.stac-item.json"
    items.append(Item.from_file(url))

In [ ]:
# Bounding box covering only viti levu
bbox = [177.20, -18.30, 178.75, -17.25]

data = load(
    items,
    chunks={"time": 1, "x": 2500, "y": 2500},
    bbox=bbox,
    measurements=["evi2"]
).compute()

data

In [ ]:
evi2 = data.evi2.clip(0, 365)

evi2.plot.hist(bins=100)

In [ ]:
m = data.evi2.isel(time=0).odc.explore()

m

In [23]:
m.save("viti_levu_all.html")